In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
import joblib

X_train = np.load("../data/gold/X_train_esm2.npy")
y_train = np.load("../data/gold/y_train_labels.npy")
X_test = np.load("../data/gold/X_test_esm2.npy")
y_pred_nn = np.load("../data/gold/y_pred_nn.npy")
top_terms = joblib.load("../models/top_terms_1500.pkl")
test_ids = np.load("../data/gold/test_protein_ids.npy", allow_pickle=True)

print("🌲 Trenowanie XGBoost na GPU...")
clf = MultiOutputClassifier(xgb.XGBClassifier(
    tree_method='gpu_hist', n_estimators=100, max_depth=5, device="cuda"
))
clf.fit(X_train, y_train)

print("🧠 Predykcja XGBoost...")
y_pred_xgb = np.array([p[:, 1] for p in clf.predict_proba(X_test)]).T

# Ensemble
y_final = (0.7 * y_pred_nn) + (0.3 * y_pred_xgb)

print("💾 Zapisywanie submission...")
with open("submission.tsv", "w") as f:
    for i, prot_id in enumerate(test_ids):
        row = y_final[i]
        indices = np.where(row > 0.05)[0] # Próg 0.05 dla lekkości pliku
        for idx in indices:
            f.write(f"{prot_id}\t{top_terms[idx]}\t{row[idx]:.3f}\n")
print("✅ Wszystko gotowe! Plik submission.tsv wygenerowany.")